In [86]:
!curl localhost:8501/v1/models/face-mask-detection-serving 

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}


In [81]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from imutils import paths
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
import json
import requests

pathname = "testset"
imagePaths = list(paths.list_images(pathname))

In [82]:
MAP_CHARACTERS= {1: 'No Mask',
                0: 'Mask'}

In [83]:
# Apply the same preprocessing as during training (resize and rescale)
image = tf.io.decode_image(open('testset/NoMask/frame_2021_12_21_16_10_05_0_1.jpg', 'rb').read(), channels=3)
image = tf.image.resize(image, [224, 224])
image = image/255.
# Convert the Tensor to a batch of Tensors and then to a list
image_tensor = tf.expand_dims(image, 0)
image_tensor = image_tensor.numpy().tolist()
# Define the endpoint with the format: http://localhost:8501/v1/models/MODEL_NAME:predict
endpoint = "http://localhost:8501/v1/models/face-mask-detection-serving:predict"

# Prepare the data that is going to be sent in the POST request
json_data = json.dumps({
  "instances": image_tensor
})

# Send the request to the Prediction API
headers = {"content-type": "application/json"}
response = requests.post(url="http://localhost:8501/v1/models/face-mask-detection-serving:predict", data=json_data, headers=headers)

# Retrieve the highest probablity index of the Tensor (actual prediction)
prediction = tf.argmax(response.json()['predictions'], 1).numpy()[0]
# print(MAP_CHARACTERS[prediction.numpy()])
# >>> "homer_simpson"
MAP_CHARACTERS[prediction]

'No Mask'

In [84]:
import grpc

from tensorflow_serving.apis import predict_pb2, prediction_service_pb2_grpc

# Apply the same preprocessing as during training (resize and rescale)
image = tf.io.decode_image(open('testset/NoMask/frame_2021_12_17_10_43_04_0.jpg', 'rb').read(), channels=3)
image = tf.image.resize(image, [224, 224])
image = image/255.

# Convert the Tensor to a batch of Tensors and then to a list
image_tensor = tf.expand_dims(image, 0)
image_tensor = image_tensor.numpy().tolist()

# Optional: define a custom message lenght in bytes
MAX_MESSAGE_LENGTH = 20000000

# Optional: define a request timeout in seconds
REQUEST_TIMEOUT = 5

# Open a gRPC insecure channel
channel = grpc.insecure_channel(
    "localhost:8500",
    options=[
        ("grpc.max_send_message_length", MAX_MESSAGE_LENGTH),
        ("grpc.max_receive_message_length", MAX_MESSAGE_LENGTH),
    ],
)

# Create the PredictionServiceStub
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

# Create the PredictRequest and set its values
req = predict_pb2.PredictRequest()
req.model_spec.name = 'face-mask-detection-serving'
req.model_spec.signature_name = ''

# Convert to Tensor Proto and send the request
# Note that shape is in NHWC (num_samples x height x width x channels) format
tensor = tf.make_tensor_proto(image_tensor)
req.inputs["input_1"].CopyFrom(tensor)  # Available at /metadata

# Send request
response = stub.Predict(req, REQUEST_TIMEOUT)

# Handle request's response
output_tensor_proto = response.outputs["dense_1"]  # Available at /metadata
shape = tf.TensorShape(output_tensor_proto.tensor_shape)

result = tf.reshape(output_tensor_proto.float_val, shape)
result = tf.argmax(result, 1).numpy()[0]
print(MAP_CHARACTERS[result])


No Mask
